In [22]:
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
from scipy.special import binom

class  hidalgo():

	def __init__(self, metric = 'euclidean', K = 1, zeta=0.8, q=3, Niter=10, Nreplicas=1,burn_in=0.5,a=np.ones(2),b=np.ones(2),c=np.ones(2),f=np.ones(2)):

		a=np.ones(K)
		b=np.ones(K)
		c=np.ones(K)
		f=np.ones(K)

		self.metric = metric
		self.K = K
		self.zeta = zeta
		self.q = q
		self.Niter=Niter
		self.burn_in=burn_in
		self.Nreplicas=Nreplicas
		self.a=a
		self.b=b
		self.c=c
		self.f=f

	def _fit(self,X):

		q=self.q
		K=self.K
		Niter=self.Niter
		zeta=self.zeta
		Nreplicas=self.Nreplicas
		a=self.a
		b=self.b
		c=self.c
		f=self.f
		burn_in=self.burn_in

		assert isinstance(X,np.ndarray), "X should be a numpy array"
		assert len(np.shape(X))==2, "X should be a two-dimensional numpy array"

		N,d = np.shape(X)

		if self.metric!='predefined':
			nbrs = NearestNeighbors(n_neighbors=q+1, algorithm='ball_tree',metric=self.metric).fit(X)
			distances, indicesIn = nbrs.kneighbors(X)

			nbrmat=np.zeros((N,N))

			for n in range(q):
				nbrmat[indicesIn[:,0],indicesIn[:,n+1]]=1

			nbrcount=np.sum(nbrmat,axis=0)
			indicesOut=np.where(nbrmat.T)[1]
			indicesTrack=np.cumsum(nbrcount)
			indicesTrack=np.append(0,indicesTrack[:-1])
			
		else:
			distances = np.sort(X)[:,:q+1]
			indicesIn = np.argsort(X)[:,:q+1]

			nbrmat=np.zeros((N,N))

			for n in range(q):
				nbrmat[indicesIn[:,0],indicesIn[:,n+1]]=1

			nbrcount=np.sum(nbrmat,axis=0)
			indicesOut=np.where(nbrmat.T)[1]
			indicesTrack=np.cumsum(nbrcount)
			indicesTrack=np.append(0,indicesTrack[:-1])

		mu = np.divide(distances[:,2],distances[:,1])

		fixed_Z=0;
		use_Potts=1;
		estimate_zeta=0;
		sampling_rate=2;
		Nsamp=np.floor((Niter-np.ceil(burn_in*Niter))/sampling_rate).astype(int)
		self.Nsamp=Nsamp
		Npar=N+2*K+2+1*(estimate_zeta);

		sampling=2*np.ones(Nsamp*Npar);
		bestsampling=np.zeros((Nsamp,Npar));

		indicesIn=indicesIn[:,1:]
		indicesIn=np.reshape(indicesIn,(N*q,))

		maxlik=-1.E10

		r=1
		return (Niter,K,fixed_Z,use_Potts,estimate_zeta,q,zeta,sampling_rate,burn_in,r,mu,indicesIn.astype(float),indicesOut.astype(float),nbrcount,indicesTrack,a,b,c,f,sampling)
			

		for r in range(Nreplicas):
			print(Niter,K,fixed_Z,use_Potts,estimate_zeta,q,zeta,sampling_rate,burn_in,r,mu,indicesIn.astype(float),indicesOut.astype(float),nbrcount,indicesTrack,a,b,c,f,sampling)
			sampling=np.reshape(sampling,(Nsamp,Npar))	
			lik=np.mean(sampling[:,-1],axis=0)
			if(lik>maxlik): 
				bestsampling=sampling
			sampling=np.reshape(sampling,(Nsamp*Npar,))	

		return bestsampling



	def fit(self, X):
		N=np.shape(X)[0]
		
		return self._fit(X)
		
		sampling=self._fit(X)
		K=self.K

		self.d_=np.mean(sampling[:,:K],axis=0)
		self.derr_=np.std(sampling[:,:K],axis=0)
		self.p_=np.mean(sampling[:,K:2*K],axis=0)
		self.perr_=np.std(sampling[:,K:2*K],axis=0)
		self.lik_=np.mean(sampling[:,-1],axis=0)
		self.likerr_=np.std(sampling[:,-1],axis=0)
		
		Pi=np.zeros((K,N))

		for k in range(K):
			Pi[k,:]=np.sum(sampling[:,2*K:2*K+N]==k,axis=0)

		self.Pi=Pi/self.Nsamp
		Z=np.argmax(Pi,axis=0);
		pZ=np.max(Pi,axis=0);
		Z=Z+1
		Z[np.where(pZ<0.8)]=0
		self.Z=Z


# partition function Z
def Zpart(N, N1, zeta, q):
    s=0
    for q1 in range(q+1):
        s += binom(N1-1,q1) * binom(N-N1,q-q1) * zeta**(q1) * (1-zeta)**(q-q1)
    
    return s

#################### scenario ######################

# get model
K=2

# generate dataset
N=5
d1=1
d2=3

np.random.seed(10002)
X=np.zeros((2*N,6))

for j in range(d1):
	X[:N,j]= np.random.normal(0,3,N)

for j in range(d2):
	X[N:,j]= np.random.normal(2,1,N)


model=hidalgo(K=K)
Niter,K,fixed_Z,use_Potts,estimate_zeta,q,zeta,sampling_rate,burn_in,r,mu,indicesIn,indicesOut,nbrcount,indicesTrack,a,b,c,f,sampling = model.fit(X)

# name conflicts
N_ = mu.size
MU = mu
Iin = indicesIn.astype(int)
Iout = indicesOut.astype(int)
Iout_count = nbrcount.astype(int)
Iout_track = indicesTrack.astype(int)


print('int Niter = ',Niter,';', sep='')
print('const int K = ',K,';', sep='')
print('bool fixed_Z = ',fixed_Z,';', sep='')
print('bool use_Potts = ',use_Potts,';', sep='')
print('bool estimate_zeta = ',estimate_zeta,';', sep='')
print('int q = ',q,';', sep='')
print('vector <double> MU = {', ', '.join(mu.astype(str)),'};', sep='')
print('vector<int> Iin = {',', '.join(indicesIn.astype(int).astype(str)),'};',sep='')
print('vector<int> Iout = {',', '.join(indicesOut.astype(int).astype(str)),'};',sep='')
print('int Iout_count[] = {', ', '.join(nbrcount.astype(int).astype(str)),'};', sep='')
print('int Iout_track[] = {', ', '.join(indicesTrack.astype(int).astype(str)),'};', sep='')
print('double a[] = {',', '.join(a.astype(str)),'};', sep='')
print('double b[] = {',', '.join(b.astype(str)),'};', sep='')
print('double c[] = {',', '.join(c.astype(str)),'};', sep='')
print('double f[] = {',', '.join(f.astype(str)),'};', sep='')
print('double zeta = ',zeta,';', sep='')
print('vector <double> sampling = {',', '.join(sampling.astype(str)),'};', sep='')
print('int sampling_rate = ',sampling_rate,';', sep='')
print('int replica = ',r,';', sep='')

# burn_in is hardcoded in c++ code
# replaced N=MU.size();
print('const int N = ',mu.size,';', sep='')


int Niter = 10;
const int K = 2;
bool fixed_Z = 1;
bool use_Potts = 1;
bool estimate_zeta = 0;
int q = 3;
vector <double> MU = {1.4647219996062466, 5.409740134317853, 2.4647219996062466, 4.7817501050892925, 1.1746064081642558, 1.9589087445409834, 1.058637057660818, 1.0631338562261194, 1.8504063601074203, 1.2806526635220237};
vector<int> Iin = {2, 4, 7, 3, 9, 4, 0, 4, 7, 1, 4, 9, 0, 9, 3, 8, 6, 9, 8, 5, 9, 5, 8, 9, 5, 6, 7, 5, 7, 4};
vector<int> Iout = {2, 4, 3, 0, 1, 4, 0, 1, 2, 3, 9, 6, 7, 8, 9, 5, 8, 0, 2, 8, 9, 5, 6, 7, 1, 3, 4, 5, 6, 7};
int Iout_count[] = {2, 1, 1, 2, 5, 4, 2, 4, 3, 6};
int Iout_track[] = {0, 2, 3, 4, 6, 11, 15, 17, 21, 24};
double a[] = {1.0, 1.0};
double b[] = {1.0, 1.0};
double c[] = {1.0, 1.0};
double f[] = {1.0, 1.0};
double zeta = 0.8;
vector <double> sampling = {2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0};
int sampling_rate = 2;
int replica = 

In [55]:
sampling_check = [3.04431, 1.56489, 0.332044, 0.667956, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, -18.4604, -4.99662, 1.81112, 0.892297, 0.434086, 0.565914, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, -18.1564, -4.69259]
len(sampling_check)

32

In [23]:
# paste the above into `gibbs_katie_print_random.cpp` and retrive
# the randomly generated numbers for run as `random_numbers.csv`

import pandas as pd
random_list = pd.read_csv('random_numbers.csv', header=None).values.tolist()[0]

#### INITIALIZE PARAMETERS ###############################

# params to initialise
V = np.empty(shape=K)
NN = np.empty(shape=K)
d = np.empty(shape=K)
p = np.empty(shape=K)
a1 = np.empty(shape=K)
b1= np.empty(shape=K)
c1 = np.empty(shape=K)
Z = np.empty(shape=N_, dtype=int)
f1 = np.empty(shape=2)

for k in range(K):
    V[k]=0 
    NN[k]=0
    d[k]=1
    sampling = np.append(sampling, d[k])

for k in range(K):
    p[k]=1./K
    sampling = np.append(sampling, p[k])

pp=(K-1)/K

for i in range(N_):
    z = random_list.pop(0)
    if fixed_Z==False:
        #z = int(np.floor(random.random()*K))
        Z[i]=z
    else:
        Z[i]=0

    V[Z[i]]=V[Z[i]]+np.log(MU[i])
    NN[Z[i]]+=1
    sampling = np.append(sampling, Z[i])

for k in range(K):
    a1[k]=a[k]+NN[k]
    b1[k]=b[k]+V[k]
    c1[k]=c[k]+NN[k]

N_in=0

for i in range(N_):
    k=Z[i]

    for j in range(q):
        index = Iin[q*i+j]
        if Z[index]==k: N_in +=1


f1[0]=f[0]+N_in; 
f1[1]=f[1]+N_*q-N_in

sampling = np.append(sampling, 0)
sampling = np.append(sampling, 0)

In [24]:
N_

10

In [25]:
print(V, NN, d, p, a1, b1, c1, Z, 'f1',f1, N_in, pp, sampling)

[6.35105124 0.        ] [10.  0.] [1. 1.] [0.5 0.5] [11.  1.] [7.35105124 1.        ] [11.  1.] [0 0 0 0 0 0 0 0 0 0] f1 [31.  1.] 30 0.5 [2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.  1.  0.5 0.5
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]


In [26]:


########## check scenario 1 ###########

sampling2 = [
2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0.5, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ]

assert all(sampling == sampling2)

#######################################

#for it in range(Niter):

it = 0
#### SAMPLING d ###############################
for k in range(K):
    stop = False

    while stop ==False:

        #r1 = random.random()*200 # random sample for d[k]
        #r2 = random.random() # random number for accepting

        r1 = random_list.pop(0)
        r2 = random_list.pop(0)

        rmax = (a1[k]-1)/b1[k]

        if (a1[k]-1>0): 
            frac = np.exp(-b1[k]*(r1-rmax)-(a1[k]-1)*(np.log(rmax)-np.log(r1))) 
        else:
            frac = np.exp(-b1[k]*r1)

        if (frac>r2):
            stop=True
            if(it%sampling_rate==0 and it>= Niter*burn_in):
                sampling = np.append(sampling, r1)
            d[k]=r1


#### SAMPLING p ###############################
for k in range(K-1):
    stop = False

    while stop ==False:

        #r1 = random.random() # random sample for p[k]
        #r2 = random.random() # random number for accepting

        r1 = random_list.pop(0)
        r2 = random_list.pop(0)

        rmax = (c1[k]-1)/(c1[k]-1+c1[K-1]-1)
        frac = ((r1/rmax)**(c1[k]-1))*(((1-r1)/(1-rmax))**(c1[K-1]-1))

        if(frac>r2):
            stop=True
            r1=r1*(1.-pp+p[k])
            p[K-1]+=p[k]-r1
            pp-=p[k]-r1
            p[k]=r1
            if(it%sampling_rate==0 and it>= Niter*burn_in):
                sampling = np.append(sampling, r1)

if(it%sampling_rate==0 and it>= Niter*burn_in):
    sampling = np.append(sampling, (1-pp))


#### SAMPLING zeta ###############################
stop=False
maxval=-100000
mx=0
l=0

if bool(use_Potts)==True and bool(estimate_zeta)==True:
    for l in range(10):
        zeta1=0.5+0.05*l
        ZZ = np.empty((K,0))
        for k in range(K):
            ZZ =np.append(ZZ, Zpart(N_, NN[k], zeta1, q))
        h = 0
        for k in range(K):
            h=h+NN[k]*np.log(ZZ[k])
        
        val = (f1[0]-1)*np.log(zeta1)+(f1[1]-1)*np.log(1-zeta1)-h

        if(val > maxval):
            maxval=val # found max val for below frac
            mx=zeta1

    while stop == False:
        #r1 = random.random() # random sample for zeta
        #r2 = random.random() # random number for accepting

        r1 = random_list.pop(0)
        r2 = random_list.pop(0)

        ZZ = np.empty((K,0))
        for k in range(K):
            ZZ =np.append(ZZ, Zpart(N_, NN[k], r1, q))
        h = 0
        for k in range(K):
            h=h+NN[k]*np.log(ZZ[k])
        
        val = (f1[0]-1)*np.log(r1)+(f1[1]-1)*np.log(1-r1)-h
        frac = np.exp(val - maxval)

        if frac > r2:
            stop = True
            if it > 0: zeta = r1

#### SAMPLING Z ###############################

for i in range(N_):

    if fixed_Z == True: break

    if abs((zeta-1)<1E-5):
        if(it%sampling_rate==0 and it>= Niter*burn_in):
            sampling = np.append(sampling, Z[i])
            continue
    
    stop = False
    prob = np.empty(shape=K)
    gg = np.empty(shape=K)
    norm = 0
    gmax = 0

    for k1 in range(K):
        g = 0
        if use_Potts == True:
            n_in = 0
            for j in range(q):
                index = int(Iin[q*i+j])
                if Z[index]==k1: n_in=n_in +1.
            m_in = 0
            for j in range(int(Iout_count[i])):
                index = int(Iout[Iout_track[i] + j])
                if index > -1 and Z[index] == k1: m_in = m_in + 1.

            g = (n_in+m_in)*np.log(zeta/(1-zeta))-np.log(Zpart(N_,NN[k1],zeta,q))
            g=g+np.log(Zpart(N_,NN[k1]-1,zeta,q)/Zpart(N_,NN[k1],zeta,q))*(NN[k1]-1)

        if g > gmax: gmax=g
        gg[k1]=g

    for k1 in range(K):
        gg[k1]=np.exp(gg[k1]-gmax)

    for k1 in range(K):
        prob[k1]=p[k1]*d[k1]*MU[i]**(-(d[k1]+1))*gg[k1]
        norm+=prob[k1]

    for k1 in range(K):
        prob[k1]=prob[k1]/norm
    
    while_loop_count = 0
    while stop == False:

        while_loop_count += 1
        if while_loop_count > 10000: break

        #r1 = int(np.floor(random.random()*K))
        #r2 = random.random()

        r1 = random_list.pop(0)
        r2 = random_list.pop(0)

        if prob[r1] > r2:
            stop = True
            if(it%sampling_rate==0 and it>= Niter*burn_in):
                sampling = np.append(sampling, r1)
            NN[Z[i]]-=1
            a1[Z[i]]-=1
            c1[Z[i]]-=1
            V[Z[i]]-=np.log(MU[i])
            b1[Z[i]]-=np.log(MU[i])
            Z[i]=r1
            NN[Z[i]]+=1
            a1[Z[i]]+=1
            c1[Z[i]]+=1
            V[Z[i]]+=np.log(MU[i])
            b1[Z[i]]+=np.log(MU[i])

#### updating prior on zeta ###############################
N_in=0
for i in range(N_):
    k=Z[i]

    for j in range(q):
        index=int(Iin[q*i+j])

        if(Z[index]==k): N_in=N_in+1.0


f1[0]=f[0]+N_in
f1[1]=f[1]+N_*q-N_in


#### likelihood ###############################
lik0=0
for i in range(N_):
    lik0=lik0+np.log(p[Z[i]])+np.log(d[Z[i]])-(d[Z[i]]+1)*np.log(MU[i])
    lik1=lik0+np.log(zeta/(1-zeta))*N_in

for k1 in range(K):
    lik1=lik1-NN[k1]*np.log(Zpart(N_, NN[k1], zeta, q))

if(it%sampling_rate==0 and it>= Niter*burn_in):
    sampling = np.append(sampling, lik0) 
    sampling = np.append(sampling, lik1)


C:\Users\n10907700\AppData\Local\Temp\ipykernel_13880\2346713082.py:52: RuntimeWarning: divide by zero encountered in double_scalars
  frac = ((r1/rmax)**(c1[k]-1))*(((1-r1)/(1-rmax))**(c1[K-1]-1))


In [37]:
sampling_check = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,1.4907,0.34657,0.95813,0.0418701,-12.2538,-8.27885,1.73127,0.263247,0.879096,0.120904,-13.1465,-9.17152,]
    

In [39]:
 sampling_check = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,1.4907,0.34657,0.95813,0.0418701,-12.2538,-8.27885,1.73127,0.263247,0.879096,0.120904,-13.1465,-9.17152,]
    

60

In [42]:
sampling_check = [1,1,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,1.4907,0.34657,0.95813,0.0418701,-12.2538,-8.27885,1.73127,0.263247,0.879096,0.120904,-13.1465,-9.17152,]
len(sampling_check)

60

In [43]:
len([2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])

32

10

In [ ]:
# # checks
a1 = [6., 6.]
b1 = [2.76597, 11.2779]
c1 = [6., 6.]

# initialised 
d = [1,1]

# most outer loop, total iterations
it=0

#### SAMPLING d ###############################
for k in range(K):
    stop = False

    while stop ==False:

        r1 = random.random()*200 # random sample for d[k]
        r2 = random.random() # random number for accepting

        # # check 1
        # k=1
        # r1 = 0.263247 
        # r2 = 0.476955 #frac should be 0.56262, correct

        # # check 2
        # k =0
        # r1=3.19551
        # r2=0.339335 #frac should be 0.37151, correct

        rmax = (a1[k]-1)/b1[k]

        if (a1[k]-1>0): 
            frac = np.exp(-b1[k]*(r1-rmax)-(a1[k]-1)*(np.log(rmax)-np.log(r1))) 
        else:
            frac = np.exp(-b1[k]*r1)


        if (frac>r2):
            stop=True
            if(it%sampling_rate==0 and it>= Niter*burn_in):
                sampling = np.append(sampling, r1)
            d[k]=r1



In [ ]:
par = K

# initialised 
p = [0.5, 0.5]
pp = 0.5

#### SAMPLING p ###############################
for k in range(K-1):
    stop = False

    while stop ==False:

        r1 = random.random() # random sample for p[k]
        r2 = random.random() # random number for accepting

        # # # check 1
        # k = 0
        # r1 = 0.507411 
        # r2 = 0.901197 
        # # frac 0.998902, correct

        # # # check 2
        # k = 0
        # r1 = 0.644791
        # r2 = 0.190724
        # #frac 0.645378, correct

        rmax = (c1[k]-1)/(c1[k]-1+c1[K-1]-1)
        frac = ((r1/rmax)**(c1[k]-1))*(((1-r1)/(1-rmax))**(c1[K-1]-1))

        if(frac>r2):
            stop=True
            r1=r1*(1.-pp+p[k])
            p[K-1]+=p[k]-r1
            pp-=p[k]-r1
            p[k]=r1
            if(it%sampling_rate==0 and it>= Niter*burn_in):
                sampling = np.append(sampling, r1)

if(it%sampling_rate==0 and it>= Niter*burn_in):
    sampling = np.append(sampling, (1-pp))


In [ ]:
NN = [6, 4] # for it = 0
f1 = [1, 31]
N_ = mu.size # name conflict with python's N !!

stop=False
maxval=-100000
mx=0
l=0

# overwrite default variable to test this section
estimate_zeta=1

#### SAMPLING zeta ###############################
if bool(use_Potts)==True and bool(estimate_zeta)==True:
    for l in range(10):
        zeta1=0.5+0.05*l
        ZZ = np.empty((K,0))
        for k in range(K):
            ZZ =np.append(ZZ, Zpart(N_, NN[k], zeta1, q))
        h = 0
        for k in range(K):
            h=h+NN[k]*np.log(ZZ[k])
        
        val = (f1[0]-1)*np.log(zeta1)+(f1[1]-1)*np.log(1-zeta1)-h
        
        # # test
        # ZZ is 10.5 10.5 
        # it 0 l 0 val -44.3082
        # ZZ is 10.8145 9.451 
        # it 0 l 1 val -47.225
        # ZZ is 11.056 8.408 
        # it 0 l 2 val -50.4233
        # ZZ is 11.2215 7.377 
        # it 0 l 3 val -53.9951
        # ZZ is 11.308 6.364 
        # it 0 l 4 val -58.0749
        # ZZ is 11.3125 5.375 
        # it 0 l 5 val -62.8713
        # ZZ is 11.232 4.416 
        # it 0 l 6 val -68.7367
        # ZZ is 11.0635 3.493 
        # it 0 l 7 val -76.3386
        # ZZ is 10.804 2.612 
        # it 0 l 8 val -87.1975
        # ZZ is 10.4505 1.779 
        # it 0 l 9 val -106.256
        
        # print('ZZ is', ZZ)
        # print("it ", it, " l ", l, " val ", val)
        ## correct!

        if(val > maxval):
            maxval=val # found max val for below frac
            mx=zeta1

    while stop == False:
        r1 = random.random() # random sample for zeta
        r2 = random.random() # random number for accepting 

        ZZ = np.empty((K,0))
        for k in range(K):
            ZZ =np.append(ZZ, Zpart(N_, NN[k], r1, q))
        h = 0
        for k in range(K):
            h=h+NN[k]*np.log(ZZ[k])
        
        val = (f1[0]-1)*np.log(r1)+(f1[1]-1)*np.log(1-r1)-h
        frac = np.exp(val - maxval)

        if frac > r2:
            stop = True
            if it > 0: zeta = r1

estimate_zeta=0

In [289]:
Z = np.array([1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ])
V = np.array([6.10573, 0.833582, ])

NN = [6., 4., ]
f1 = [1., 31., ]
pp = 0.5
par = K

# from recent print, after samples
d = [0.889266, 1.54721, ]
p = [0.390508, 0.609492, ]
pp = 0.390508
zeta = 0.8

# renamed in c++
Iin = indicesIn.astype(int)
Iout = indicesOut.astype(int)
Iout_count = nbrcount.astype(int)
Iout_track = indicesTrack.astype(int)
MU = mu

#### SAMPLING Z ###############################

for i in range(N_):

    if fixed_Z == True: break

    if abs((zeta-1)<1E-5):
        if(it%sampling_rate==0 and it>= Niter*burn_in):
            sampling = np.append(sampling, Z[i])
            continue
    
    stop = False
    prob = np.empty(shape=K)
    gg = np.empty(shape=K)
    norm = 0
    gmax = 0

    for k1 in range(K):
        g = 0
        if use_Potts == True:
            n_in = 0
            for j in range(q):
                index = int(Iin[q*i+j])
                if Z[index]==k1: n_in=n_in +1.
            m_in = 0
            for j in range(int(Iout_count[i])):
                index = int(Iout[Iout_track[i] + j])
                if index > -1 and Z[index] == k1: m_in = m_in + 1.

            g = (n_in+m_in)*np.log(zeta/(1-zeta))-np.log(Zpart(N_,NN[k1],zeta,q))
            g=g+np.log(Zpart(N_,NN[k1]-1,zeta,q)/Zpart(N_,NN[k1],zeta,q))*(NN[k1]-1)

        if g > gmax: gmax=g
        gg[k1]=g

    for k1 in range(K):
        gg[k1]=np.exp(gg[k1]-gmax)

    for k1 in range(K):
        prob[k1]=p[k1]*d[k1]*MU[i]**(-(d[k1]+1))*gg[k1]
        norm+=prob[k1]

    for k1 in range(K):
        prob[k1]=prob[k1]/norm
    

    while_loop_count = 0
    while stop == False:

        while_loop_count += 1
        if while_loop_count > 10000: break


        r1 = int(np.floor(random.random()*K))
        r2 = random.random()

        ## test ##
        if i ==0:
            r1 = 1
            r2 = 0.215729
        elif i ==1:
            r1 = 1
            r2 = 0.259906
        elif i == 2:
            r1 = 0
            r2 = 0.749929

        if prob[r1] > r2:
            #print("i ", i, " r1 ", r1, " r2 ", r2, " gg is ", gg, " norm ", norm, " gmax ", gmax, " prob[r1] ", prob[r1])

## test
# 0 r1 1 r2 0.215729 gg is 1 0.264008 norm 0.516878 gmax 0.936204 prob[r1] 0.407025 
# i 1 r1 1 r2 0.259906 gg is 0.946941 1 norm 0.00584545 gmax 0.990723 prob[r1] 0.374425 
# i 2 r1 0 r2 0.749929 gg is 1 0.015625 norm 0.328133 gmax 0.196192 prob[r1] 0.960594 

            stop = True
            if(it%sampling_rate==0 and it>= Niter*burn_in):
                sampling = np.append(sampling, r1)
            NN[Z[i]]-=1
            a1[Z[i]]-=1
            c1[Z[i]]-=1
            V[Z[i]]-=np.log(MU[i])
            b1[Z[i]]-=np.log(MU[i])
            Z[i]=r1
            NN[Z[i]]+=1
            a1[Z[i]]+=1
            c1[Z[i]]+=1
            V[Z[i]]+=np.log(MU[i])
            b1[Z[i]]+=np.log(MU[i])


In [292]:
#### updating prior on zeta ###############################

#print(Iin)
#print(Z)

N_in=0

for i in range(N_):
    k=Z[i]

    for j in range(q):
        index=int(Iin[q*i+j])

        if(Z[index]==k): N_in=N_in+1.0


f1[0]=f[0]+N_in
f1[1]=f[1]+N_*q-N_in

# check [26, 6], correct
#print(f1)



[1 3 4 4 0 7 8 9 7 0 1 4 1 0 7 7 8 9 9 8 5 8 4 1 7 9 5 8 7 5]
[1 1 0 1 1 0 0 1 0 0]
[26.0, 6.0]


In [298]:
#### likelihood ###############################

# check 1
# p = [0.390508, 0.609492 ]
# d = [0.889266, 1.54721] 

lik0=0

for i in range(N_):
    lik0=lik0+np.log(p[Z[i]])+np.log(d[Z[i]])-(d[Z[i]]+1)*np.log(MU[i])
    lik1=lik0+np.log(zeta/(1-zeta))*N_in

for k1 in range(K):
    lik1=lik1-NN[k1]*np.log(Zpart(N_, NN[k1], zeta, q))

print(lik0, lik1)
#-22.7364 -7.88629, correct
		

-22.736391161726612 -7.8862878211361185


In [ ]:
## STEP 1
# re-write c++ as python, cross-check outputs
    # sampling d, p, zeta, Z, prior, likelihood 
    # initialise variables
### DONE! ###


## STEP 2
# write as python script, check results
    # multiple scenarios (parameter sets)
    # (check order of sampling from c++ -> python)




## STEP 3
# re-write python code, optimise
    # detailed descriptions, reference eqns in paper

In [46]:
fixed_Z=1;
use_Potts=1;
estimate_zeta=0;
sampling_rate=2;
Nsamp=np.floor((Niter-np.ceil(burn_in*Niter))/sampling_rate).astype(int)
Npar=N_+2*K+2+1*(estimate_zeta);

In [50]:
d[K], p[K-1], 1-pp, zeta(estimate_zeta==true), Z[N], ? overrides in c, lik0, lik1

32

In [52]:
2*K + 1*(estimate_zeta) + 2 + N


len(sampling)

48

In [49]:
estimate_zeta

0

In [54]:
len([3.04431, 1.56489, 0.332044, 0.667956, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, -18.4604, -4.99662, 1.81112, 0.892297, 0.434086, 0.565914, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, -18.1564, -4.69259])

32